In [2]:
import pickle
import pandas as pd

from nltk import word_tokenize
from nltk.tokenize import sent_tokenize

from data import *

In [21]:
f = open('data/CommonNounPol.pickle', 'rb')
common_nouns_policy = pickle.load(f)
f.close()

#Merge all identified policy schemes into one list
policy_schemes = []

f = open('data/NDTVPolicySchemes.pickle', 'rb')
ndtv_policy_schemes = pickle.load(f)
f.close()
f = open('data/HinduPolNames.pickle', 'rb')
hindu_policy_schemes = pickle.load(f)
f.close()
f = open('data/HTPolNames.pickle', 'rb')
ht_policy_schemes = pickle.load(f)
f.close()
f = open('data/IndiaNewsPolNames.pickle', 'rb')
india_news_policy_schemes = pickle.load(f)
f.close()

for pol in ndtv_policy_schemes:
    if pol not in policy_schemes:
        policy_schemes.append(pol)
for pol in hindu_policy_schemes:
    if pol not in policy_schemes:
        policy_schemes.append(pol)
for pol in ht_policy_schemes:
    if pol not in policy_schemes:
        policy_schemes.append(pol)
for pol in india_news_policy_schemes:
    if pol not in policy_schemes:
        policy_schemes.append(pol)


In [23]:
f = open('data/AllPolSchemes.pickle', 'wb')
pickle.dump(policy_schemes, f)
f.close()

In [24]:
f = open('data/UpdNDTVNewsText', 'rb')
ndtv_articles = pickle.load(f)
f.close()

f = open('data/UpdHinduNewsText', 'rb')
hindu_articles = pickle.load(f)
f.close()

f = open('data/UpdHTNewsText', 'rb')
ht_articles = pickle.load(f)
f.close()

f = open('data/UpdIndiaNewsText', 'rb')
indiaNews_articles = pickle.load(f)
f.close()

In [25]:
#Get one common dict with all articles 
idx = 0
all_articles = {}
for _, val in ndtv_articles.items():
    all_articles[idx] = {'Newspaper': 'NDTV', 'Date': val['Date'], 'Text': val['Text']}
    idx += 1

for _, val in hindu_articles.items():
    all_articles[idx] = {'Newspaper': 'Hindu', 'Date': val['Date'], 'Text': val['Text']}
    idx += 1

for _, val in ht_articles.items():
    all_articles[idx] = {'Newspaper': 'Hindustan Times', 'Date': val['Date'], 'Text': val['Text']}
    idx += 1

for _, val in indiaNews_articles.items():
    all_articles[idx] = {'Newspaper': 'India News', 'Date': val['Date'], 'Text': val['Text']}
    idx += 1

print('Total Number of articles ---->', idx)
total_num_articles = idx

Total Number of articles ----> 10023


In [26]:
f = open('data/AllArticles.pickle', 'wb')
pickle.dump(all_articles, f)
f.close()

In [27]:
def check_policy_terms(word):
    return word in common_nouns_policy

def check_policy_names(sent):
    present = False
    polNames = []
    for pol in policy_schemes:
        if pol in sent and pol not in polNames:
            present = True
            polNames.append(pol)
    return polNames, present
    

In [28]:
policy_present_df = pd.DataFrame(columns=['Newspaper', 'Date', 'Present', 'Terms', 'Names'])

In [30]:
for key, val in all_articles.items():
    mentions = []
    pol_names = []
    text = val['Text']
    policy_present_df.at[key, 'Newspaper'] = val['Newspaper']
    policy_present_df.at[key, 'Date'] = val['Date']
    policy_present_df.at[key, 'Present'] = 'False'
    policy_present_df.at[key, 'Terms'] = 'None'
    policy_present_df.at[key, 'Names'] = 'None'
    tokens = word_tokenize(text)
    for tok in tokens:
        found = check_policy_terms(tok)
        if (found == True) and (tok not in mentions):
            mentions.append(tok)
    if len(mentions) > 0:
        policy_present_df.at[key, 'Terms'] = mentions
    sentences = sent_tokenize(text)
    for sent in sentences:
        polName, present = check_policy_names(sent)
        if present == True:
            pol_names.extend(polName)
    if len(pol_names) > 0:
        policy_present_df.at[key, 'Names'] = pol_names
    if len(mentions) > 0 or len(pol_names) > 0:
        policy_present_df.at[key, 'Present'] = 'True'

In [31]:
print("Count of articles that mention policy terms and schemes ->",
      policy_present_df[(policy_present_df['Present'] == 'True')]['Newspaper'].count())

print("Count of articles that mention policy schemes ->",
      policy_present_df[(policy_present_df['Names'] != 'None')]['Newspaper'].count())

print("Count of articles that mention policy terms ->",
      policy_present_df[(policy_present_df['Terms'] != 'None')]['Newspaper'].count())
 
# dataframe of above result
policy_present_df[(policy_present_df['Present'] == 'True')]

Count of articles that mention policy terms and schemes -> 6213
Count of articles that mention policy schemes -> 2609
Count of articles that mention policy terms -> 5608


,Newspaper,Date,Present,Terms,Names
0,NDTV,2021-03-30,True,[plan],None
1,NDTV,2021-03-23,True,"[measures, drive]",None
4,NDTV,2021-02-19,True,"[programmes, programme, campaign, drives, driv...",None
5,NDTV,2021-01-25,True,[scheme],None
7,NDTV,2020-12-31,True,"[proposal, benefits, fund, plan, campaigns, sc...",None
...,...,...,...,...,...
10017,India News,2020-04-02,True,"[measures, rights]",None
10018,India News,2020-04-02,True,[care],None
10019,India News,2020-04-01,True,"[measures, care]",None
10020,India News,2020-03-30,True,[drives],None


In [33]:
print("Percentage of articles that mention policy schemes ->", (policy_present_df[(policy_present_df['Names'] != 'None')]['Newspaper'].count())/total_num_articles * 100)
print("Percentage of articles that mention policy terms ->", (policy_present_df[(policy_present_df['Terms'] != 'None')]['Newspaper'].count())/total_num_articles * 100)

Percentage of articles that mention policy schemes -> 26.030130699391403
Percentage of articles that mention policy terms -> 55.95131198244039


In [34]:
f = open('data/PolicyPresentDf.pickle', 'wb')
pickle.dump(policy_present_df, f)
f.close()